In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
np.set_printoptions(precision=3, suppress=True)

In [ ]:
df = pd.read_csv('ana_ray_class.csv')#, converters={'source_pos':pd.eval, 'antenna_pos':pd.eval})
df.head()

In [ ]:
labels = np.array(df['n_sol'])
sc_pos_r = df['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df['antenna_pos_z'].to_numpy().astype(np.float32)
features = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

In [ ]:
features  = features.astype(np.float32)
labels[labels == 2] = 1
features_train = features[:int(features.shape[0]*0.8)]
features_test = features[int(features.shape[0]*0.8):]
labels_train = labels[:int(labels.shape[0]*0.8)]
labels_test = labels[int(labels.shape[0]*0.8):]

In [ ]:
normalize = layers.Normalization()
normalize.adapt(features_train)

In [ ]:
model = keras.Sequential()
model.add(normalize)
model.add(layers.Dense(128, input_shape=(3, 0), activation='relu'))
model.add(layers.Dense(2))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              #loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
model.fit(features_train, labels_train, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(features_test,  labels_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
y = model(features_test)

In [ ]:
y2 = np.zeros((y.shape[0],))
y2[y[:,0] < y[:,1]] = 1

true_pos = y2[(y2 + labels_test) == 2]
true_neg = y2[(y2 + labels_test) == 0] + 1 
pred = labels_test - y2
false_neg = pred[pred==1]
false_pos = np.abs(pred[pred==-1])

In [ ]:

print("total", np.sum(false_pos) + np.sum(false_neg) + np.sum(true_pos) + np.sum(true_neg), labels_test.shape)
print("false negative", np.sum(false_neg), "rate", np.sum(false_neg)/labels_test.shape[0]*100,"%")
print("false positive", np.sum(false_pos), "rate", np.sum(false_pos)/labels_test.shape[0]*100,"%")
print("true negative", np.sum(true_neg), "rate", np.sum(true_neg)/labels_test.shape[0]*100,"%")
print("true positive", np.sum(true_pos), "rate", np.sum(true_pos)/labels_test.shape[0]*100,"%")

In [ ]:
print(np.sum(labels_test==0), np.sum(labels_test==1))

In [ ]:
keras.utils.plot_model(model, "solution_classifier.png")